In [ ]:
자연어 처리
문자로 된 글은 컴퓨터가 읽을 수 없으니 숫자로 바꿔준다
방법 중 하나가 bag of words, 이 안에 count벡터랑 tfidf벡터랑
이거 한 다음에 분류기(SVC, LR) 등으로 분류함

벡터랑 분류를 한 번에 하는게 Pipeline
여기다 하이퍼 파라미터 튜닝 하려면 gridSearchcv


In [3]:
import joblib
import re
import pandas as pd 
import numpy as np 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [6]:
df_raw = pd.read_csv('../static/data/advanced/labeledTrainData.tsv',
                header = 0, sep = '\t', quoting = 3)
df_raw.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [8]:
# <br>(띄어쓰기)는 공백으로 변환
df_raw['review'] = df_raw['review'].str.replace('<br />',' ')

In [9]:
# 영어 이외의 문자는 공백으로 변환
df_raw['review'] = df_raw.review.apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

In [10]:
df_raw.isna().sum()

id           0
sentiment    0
review       0
dtype: int64

In [11]:
df_raw[df_raw.review == ''].count()

id           0
sentiment    0
review       0
dtype: int64

In [12]:
review_df = df_raw.drop(['id','sentiment'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(
    review_df, df_raw.sentiment, test_size = 0.25, random_state = 2021
)

In [13]:
print(type(X_train))
X_train[:3]

<class 'pandas.core.frame.DataFrame'>


,review
14475,There s no shortage of bad dialogue in David ...
22605,This film takes what could have been a good i...
17673,Bob Clampett s Porky s Poor Fish is a so so...


In [14]:
print(type(X_test))
X_test[:3]

<class 'pandas.core.frame.DataFrame'>


,review
13895,I was expecting a B Movie French musical Aft...
20903,Disappearance is about a couple who take thei...
8539,I noticed at once that this movie really wasn...


In [15]:
df_test = pd.DataFrame(X_test, columns=['review'])
df_test['sentiment'] = y_test
df_test.head(3)

,review,sentiment
13895,I was expecting a B Movie French musical Aft...,0
20903,Disappearance is about a couple who take thei...,0
8539,I noticed at once that this movie really wasn...,1


In [20]:
df_test.to_csv('../static/data/IMDB_test.csv', index=False)

1. CountVectorizer + 로지스틱 회귀

In [21]:
pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression())
])
params = ({
    'count_vect__max_df': [100, 300, 500],
    'lr_clf__C': [1, 5, 10]
})

In [22]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1, n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  6.6min finished
{'count_vect__max_df': 500, 'lr_clf__C': 1} 0.8667199999999999


In [23]:
best_count_lr = grid_pipe.best_estimator_
pred_count_lr = best_count_lr.predict(df_test.review.values)
accuracy_score(df_test.sentiment.values, pred_count_lr)

0.87344

In [24]:
joblib.dump(best_count_lr, '../static/model/imdb_count_lr.pkl')

['../static/model/imdb_count_lr.pkl']


2. CountVectorizer + SVC

In [16]:
pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('sv_clf', SVC())
])
params = ({
    'count_vect__max_df': [100, 300, 500],
    'sv_clf__C': [0.1, 1, 10]
})

In [17]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1, n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 81.5min finished
{'count_vect__max_df': 500, 'sv_clf__C': 10} 0.8602666666666666


In [18]:
best_count_sv = grid_pipe
pred_count_sv = best_count_sv.predict(df_test.review.values)
accuracy_score(df_test.sentiment.values, pred_count_sv)

0.86688

In [19]:
joblib.dump(best_count_sv, '../static/model/imdb_count_sv.pkl')

['../static/model/imdb_count_sv.pkl']

3. TfidfVectorizer + LogisticRegression

In [17]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression())
])
params = ({
    'tfidf_vect__max_df': [100, 300, 500],
    'lr_clf__C': [0.1, 1, 10]
})

In [18]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1, n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  2.9min finished
{'lr_clf__C': 10, 'tfidf_vect__max_df': 500} 0.8776533333333333


In [19]:
best_tfidf_lr = grid_pipe.best_estimator_
pred_tfidf_lr = best_tfidf_lr.predict(df_test.review.values)
accuracy_score(df_test.sentiment.values, pred_tfidf_lr)

0.88144

In [20]:
joblib.dump(best_tfidf_lr, '../static/model/imdb_tfidf_lr.pkl')

['../static/model/imdb_tfidf_lr.pkl']

4. TfidfVectorizer + SupportVectorMachine

In [21]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('sv_clf', SVC())
])
params = ({
    'tfidf_vect__max_df': [100, 300, 500],
    'sv_clf__C': [0.1, 1, 10]
})

In [22]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1, n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 75.6min finished
{'sv_clf__C': 10, 'tfidf_vect__max_df': 500} 0.87488


In [23]:
best_tfidf_sv = grid_pipe
pred_tfidf_sv = best_tfidf_sv.predict(df_test.review.values)
accuracy_score(df_test.sentiment.values, pred_tfidf_sv)

0.87984

In [24]:
joblib.dump(best_tfidf_sv, '../static/model/imdb_tfidf_sv.pkl')

['../static/model/imdb_tfidf_sv.pkl']

왜 test_data = \[\] 를 만들고 거기에다 append를 하는가? <br>
그냥 나오는건 string이라서 <br>

pipeline은 벡터라이저 + 분류기<br>

벡터라이저에는 \['문자열','문자열','문자열'\] 로 들어가야함<br>
그렇다고 list(스트링) 하면 한 글자씩 다 잘라서 나옴<br>

예로 \['length=20인 놈','12','13'\] 이 들어가면 자동적으로 차원이 (3,45) 이런식으로
2차원이 되기때문에
분류기에 자동적으로 들어갈 수 있음<br>



### 테스트

In [25]:
index = 1000

In [26]:
df_test.iloc[index, 0]

' i say the domino principle is an enormously underappreciated film anyone who has taken the time to investigate our contemporary history of conspiracies jfk  rfk  mlk g wallace and in fact numerous others can only draw the conclusion that the author of the domino principle really knew what he was talking about roy tucker could be lee harvey oswald or james earl ray or sirhan sirhan or arthur bremer maybe even john hinkley or timothy mcveigh to mention a few the conspiracy scenario involving spies  big business and political assassinations is not really a fiction but an ominous part of our convoluted existential history god help us but the domino principle is more fact than fantasy if this causes a little loss of sleep  maybe it should don t take my word for it investigate for yourselves  '

In [27]:
test_data = []
test_data.append(df_test.iloc[index, 0])

In [28]:
label = df_test.sentiment[index]
label

0

In [29]:
model_cl = joblib.load('../static/model/imdb_count_lr.pkl')
model_cs = joblib.load('../static/model/imdb_count_sv.pkl')
model_tl = joblib.load('../static/model/imdb_tfidf_lr.pkl')
model_ts = joblib.load('../static/model/imdb_tfidf_sv.pkl')

In [30]:
pred_cl = model_cl.predict(test_data)
pred_cs = model_cs.predict(test_data)
pred_tl = model_tl.predict(test_data)
pred_ts = model_ts.predict(test_data)

In [31]:
print(label, pred_cl[0], pred_cs[0], pred_tl[0], pred_ts[0])

0 1 0 1 1
